In [ ]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", len(physical_devices))

In [ ]:
physical_devices

In [ ]:
tf.__version__

In [ ]:
from keras.backend import manual_variable_initialization
manual_variable_initialization(True)

In [ ]:
%ls

In [ ]:
%cd content/

In [ ]:
#!pip uninstall tensorflow
#!pip install tensorflow==2.15.0

In [ ]:
#Data Fetching
!pip install opencv-contrib-python
from IPython.display import clear_output
#!unzip "/content/Total Images.zip" -d "/content/"
#!unzip "/content/Total GT.zip" -d "/content/"
#!unzip "/content/Dataset.zip" -d "/content/"
!unzip "/content/img_aug.zip" -d "/content/"
!unzip "/content/labels_aug.zip" -d "/content/"
#clear_output()

In [ ]:
!rm -rf /content/Dataset/

In [ ]:
!rm -rf "/content/Full Dataset.zip"

In [ ]:
from IPython.display import clear_output

!unzip "/content/Full Dataset.zip" -d "/content/Dataset/"
clear_output()

In [ ]:
#Useful imports
#will remain the same
from tensorflow.keras.layers.experimental.preprocessing import StringLookup
from tensorflow import keras


import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
import cv2 as cv
from google.colab.patches import cv2_imshow

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
#imagesDirectory = '/content/Dataset/Total Images'
imagesDirectory = '/content/Dataset/Full Dataset/Images'

for imagePath in sorted(os.listdir(imagesDirectory)):
  if imagePath.split('.')[-1] == "tif":
    imageTif = cv.imread(imagesDirectory+"/"+imagePath, cv.IMREAD_UNCHANGED)
    #cv2_imshow(imageTif)
    newImagePath = imagePath.replace('.tif','.jpg')
    cv.imwrite(imagesDirectory+"/"+newImagePath,imageTif)
    os.remove(imagesDirectory+"/"+imagePath)

In [ ]:
#filenames_img=sorted(os.listdir('/content/Dataset/Total Images'))
#filenames_gt=sorted(os.listdir('/content/Dataset/Total GT'))

filenames_img=sorted(os.listdir('/content/Dataset/Full Dataset/Images'))
filenames_gt=sorted(os.listdir('/content/Dataset/Full Dataset/Labels'))

#filenames_imgsplit=[filename.replace('.tif', '') for filename in filenames_img]
filenames_imgsplit=[filename.replace('.jpg', '') for filename in filenames_img]
#filenames_imgsplit=[filename.replace('.bmp', '') for filename in filenames_imgsplit]

filenames_gtsplit=[filename.replace('.txt', '') for filename in filenames_gt]

print(len(filenames_imgsplit))
print(len(filenames_gtsplit))

In [ ]:
 #train test validate splitting
split_idx = int(0.8 * len(filenames_imgsplit))
train_samples = filenames_img[:split_idx]
train_samples_split = filenames_imgsplit[:split_idx]
test_samples = filenames_img[split_idx:]
test_samples_split = filenames_imgsplit[split_idx:]

val_split_idx = int(0.5 * len(test_samples))
validation_samples = test_samples[:val_split_idx]
validation_samples_split = test_samples_split[:val_split_idx]
test_samples = test_samples[val_split_idx:]
test_samples_split = test_samples_split[val_split_idx:]

assert len(filenames_imgsplit) == len(train_samples) + len(validation_samples) + len(
    test_samples
)

print(f"Total training samples: {len(train_samples)}")
print(f"Total validation samples: {len(validation_samples)}")
print(f"Total test samples: {len(test_samples)}")
print(f' Example from training dataset {train_samples_split[0]}')

In [ ]:
#get data and labels as lists
#base_path='/content/Dataset/'
#base_image_path = os.path.join(base_path, "Total Images/")
#base_GT_path = os.path.join(base_path, "Total GT/")

base_path='/content/Dataset/Full Dataset/'
base_image_path = os.path.join(base_path, "Images/")
base_GT_path = os.path.join(base_path, "Labels/")

def get_image_paths_and_labels(filenames_img, filenames_imgsplit):
    paths = []
    labels = []
    for i in range(len(filenames_imgsplit)):
        img_path = os.path.join(
            base_image_path,  filenames_img[i]
        )
        if os.path.getsize(img_path):
            paths.append(img_path)
            label_path = os.path.join(
            base_GT_path,  filenames_imgsplit[i]+'.txt'
        )
        label_file = open(label_path, "r")
        labels.append(label_file.read())

    return paths, labels

train_img_paths, train_labels = get_image_paths_and_labels(train_samples, train_samples_split)
validation_img_paths, validation_labels = get_image_paths_and_labels(validation_samples, validation_samples_split)
test_img_paths, test_labels = get_image_paths_and_labels(test_samples, test_samples_split)

In [ ]:
 # Find maximum length and the size of the vocabulary in the training data.
train_labels_cleaned = []
characters = set()
max_len = 0

for label in train_labels:
    label = label.split(" ")[-1].strip()
    for char in label:
        characters.add(char)

    max_len = max(max_len, len(label))
    train_labels_cleaned.append(label)

print("Maximum length: ", max_len)
print("Vocab size: ", len(characters))

# Check some label samples.
train_labels_cleaned[:10]

In [ ]:
def clean_labels(labels):
    cleaned_labels = []
    for label in labels:
        label = label.split(" ")[-1].strip()
        cleaned_labels.append(label)
    return cleaned_labels


validation_labels_cleaned = clean_labels(validation_labels)
test_labels_cleaned = clean_labels(test_labels)


In [ ]:
print(characters)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

# Mapping characters to integers.
char_to_num = StringLookup(vocabulary=list(sorted(characters)), mask_token=None)

# Mapping integers back to original characters.
num_to_char = StringLookup(
    vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True
)

char_to_num.get_vocabulary()

In [ ]:
def distortion_free_resize(image, img_size):
    w, h = img_size
    image = tf.image.resize(image, size=(h, w), preserve_aspect_ratio=True)

    # Check tha amount of padding needed to be done.
    pad_height = h - tf.shape(image)[0]
    pad_width = w - tf.shape(image)[1]

    # Only necessary if you want to do same amount of padding on both sides.
    if pad_height % 2 != 0:
        height = pad_height // 2
        pad_height_top = height + 1
        pad_height_bottom = height
    else:
        pad_height_top = pad_height_bottom = pad_height // 2

    if pad_width % 2 != 0:
        width = pad_width // 2
        pad_width_left = width + 1
        pad_width_right = width
    else:
        pad_width_left = pad_width_right = pad_width // 2

    image = tf.pad(
        image,
        paddings=[
            [pad_height_top, pad_height_bottom],
            [pad_width_left, pad_width_right],
            [0, 0],
        ],
    )

    image = tf.transpose(image, perm=[1, 0, 2])
    image = tf.image.flip_left_right(image)
    return image

In [ ]:
print(train_labels_cleaned)

In [ ]:
len(train_labels_cleaned)

In [ ]:
batch_size = 64
padding_token = 99
image_width = 64
image_height = 32


def preprocess_image(image_path, img_size=(image_width, image_height)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, 0)

    image = distortion_free_resize(image, img_size)

    image =  tf.where(image > 80, 255, 0)#Basic thresholding

    image = tf.cast(image, tf.float32) / 255.0

    return image


def vectorize_label(label):
    print(label)
    label = char_to_num(tf.strings.unicode_split(label, input_encoding="UTF-8"))
    length = tf.shape(label)[0]
    pad_amount = max_len - length
    label = tf.pad(label, paddings=[[0, pad_amount]], constant_values=padding_token)
    return label

label=vectorize_label(train_labels_cleaned[0])
print(label)
indices = tf.gather(label, tf.where(tf.math.not_equal(label, padding_token)))
label = tf.strings.reduce_join(num_to_char(indices))
label = label.numpy().decode("UTF-8")
print(label)

def process_images_labels(image_path, label):
    image = preprocess_image(image_path)
    label = vectorize_label(label)
    return {"image": image, "label": label}


def prepare_dataset(image_paths, labels):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels)).map(
        process_images_labels, num_parallel_calls=AUTOTUNE
    )
    return dataset.batch(batch_size).cache().prefetch(AUTOTUNE)

In [ ]:
train_ds = prepare_dataset(train_img_paths, train_labels_cleaned)
validation_ds = prepare_dataset(validation_img_paths, validation_labels_cleaned)
test_ds = prepare_dataset(test_img_paths, test_labels_cleaned)

In [ ]:
for data in test_ds.take(6):
    images, labels = data["image"], data["label"]

    _, ax = plt.subplots(4, 4, figsize=(15, 8))

    for i in range(16):
        img = images[i]
        img = tf.image.flip_left_right(img)
        img = tf.transpose(img, perm=[1, 0, 2])
        img = (img * 255.0).numpy().clip(0, 255).astype(np.uint8)
        img = img[:, :, 0]

        # Gather indices where label!= padding_token.
        label = labels[i]
        indices = tf.gather(label, tf.where(tf.math.not_equal(label, padding_token)))
        # Convert to string.
        label = tf.strings.reduce_join(num_to_char(indices))
        label = label.numpy().decode("utf-8")

        ax[i // 4, i % 4].imshow(img, cmap="gray")
        ax[i // 4, i % 4].set_title(label[::-1])
        ax[i // 4, i % 4].axis("off")


plt.show()

In [ ]:
class CTCLayer(keras.layers.Layer):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.loss_fn = keras.backend.ctc_batch_cost

    def call(self, y_true, y_pred):
        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
        input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
        label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
        self.add_loss(loss)
        #print("True y:",y_true)
        print("Pred y", y_pred)
        # At test time, just return the computed predictions.
        #return y_pred


def build_model():
    # Inputs to the model
    input_img = keras.Input(shape=(image_width, image_height, 1), name="image")
    labels = keras.layers.Input(name="label", shape=(None,))

    # First conv block.
    x = keras.layers.Conv2D(
        32,
        (3, 3),
        activation="relu",
        kernel_initializer="he_normal",
        padding="same",
        name="Conv1",
    )(input_img)
    x = keras.layers.MaxPooling2D((2, 2), name="pool1")(x)
    x= keras.layers.BatchNormalization()(x)

    #This CNN layer wasn't here originally
    x = keras.layers.Conv2D(
        64,
        (3, 3),
        activation="relu",
        kernel_initializer="he_normal",
        padding="same",
        name="Conv2",
    )(x)
    x = keras.layers.MaxPooling2D((2, 2), name="pool2")(x)
    x= keras.layers.BatchNormalization()(x)

    new_shape = ((image_width // 2), (image_height // 2) * 16)#was originally *32
    x = keras.layers.Reshape(target_shape=new_shape, name="reshape")(x)

    x = keras.layers.Dense(16, activation="relu", name="dense2")(x)
    x= keras.layers.BatchNormalization()(x)

    x = keras.layers.Bidirectional(
        keras.layers.LSTM(256, return_sequences=True, dropout=0.35))(x)
    x = keras.layers.Dense(
        len(char_to_num.get_vocabulary()) + 2, activation="softmax", name="dense3"
    )(x)

    output = CTCLayer(name="ctc_loss")(labels, x)
    #print("labels:", labels)
    #print(x)

    # Define the model.
    model = keras.models.Model(
        inputs=[input_img, labels], outputs=output, name="handwriting_recognizer"
    )
    # Optimizer.
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)
    opt = keras.optimizers.Adam(learning_rate=lr_schedule)
    # Compile the model and return.
    model.compile(optimizer=opt
                )
    return model


# Get the model.
model = build_model()
model.summary()

In [ ]:
validation_images = []
validation_labels = []

for batch in validation_ds:
    validation_images.append(batch["image"])
    validation_labels.append(batch["label"])

In [ ]:
def calculate_edit_distance(labels, predictions):
    # Get a single batch and convert its labels to sparse tensors.
    sparse_labels = tf.sparse.from_dense(labels)

    # Make predictions and convert them to sparse tensors.
    input_len = np.ones(predictions.shape[0]) * predictions.shape[1]

    predictions_decoded = keras.backend.ctc_decode(

        predictions, input_length=input_len, greedy=False, beam_width=100,
    )[0][0][:, :max_len]
    sparse_predictions =tf.sparse.from_dense(predictions_decoded)

    # Compute individual edit distances and average them out.
    edit_distances = tf.edit_distance(
        sparse_predictions, sparse_labels, normalize=False
    )
    return tf.reduce_mean(edit_distances)


class EditDistanceCallback(keras.callbacks.Callback):
    def __init__(self, pred_model):
        super().__init__()
        self.prediction_model = pred_model

    def on_epoch_end(self, epoch, logs=None):
        edit_distances = []

#         for i in range(len(validation_images)):
#             labels = validation_labels[i]
#             predictions = self.prediction_model.predict(validation_images[i],verbose=0)
#             #print(decode_batch_predictions(predictions))
# #            print("Prediction")
# #            print(predictions)
# #            print("Labels:")
# #            print(labels)
# #            print("----------------------")
#             edit_distances.append(calculate_edit_distance(labels, predictions).numpy())
        #if epoch>300:
        #  getCharacterAccuracy(test_ds)#Get accuracy on dataset after every epoch but it's very slow (needs an extra 80% of the original time)
       # print(
       #     f"\nMean edit distance for epoch {epoch + 1}: {np.mean(edit_distances):.4f}"
       # )

In [ ]:
epochs = 450  # To get good results this should be at least 50.

model = build_model()
prediction_model = keras.models.Model(
    model.get_layer(name="image").input, model.get_layer(name="dense3").output
)
edit_distance_callback = EditDistanceCallback(prediction_model)

stopping=tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=3,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

# Train the model.
history = model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=epochs,
    callbacks=[edit_distance_callback],
    shuffle=True
)

#add early stopping mechanism

In [ ]:
# A utility function to decode the output of the network.
def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    # Use greedy search. For complex tasks, you can use beam search.
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=False)[0][0][
        :, :max_len
    ]
    # Iterate over the results and get back the text.
    output_text = []
    #print(results)
    for res in results:
        res = tf.gather(res, tf.where(tf.math.not_equal(res, -1)))
        res = tf.strings.reduce_join(num_to_char(res)).numpy().decode("UTF-8")
        output_text.append(res)
    #print(output_text)

    return output_text

In [ ]:
#prediction_model = tf.keras.models.load_model('epochs1000.h5')

In [ ]:
#  Let's check results on some test samples.
for batch in test_ds.take(1):
  batch_images,batch_labels = batch["image"],batch["label"]
  _, ax = plt.subplots(4, 16, figsize=(40, 8))

  preds = prediction_model.predict(batch_images)
  pred_texts = decode_batch_predictions(preds)


  for i in range(len(batch_images)):
        img = batch_images[i]

        indices = tf.gather(batch_labels[i], tf.where(tf.math.not_equal(batch_labels[i], padding_token)))
        label = tf.strings.reduce_join(num_to_char(indices))
        label = label.numpy().decode("UTF-8")

        img = tf.image.flip_left_right(img)
        img = tf.transpose(img, perm=[1, 0, 2])
        img = (img * 255.0).numpy().clip(0, 255).astype(np.uint8)
        img = img[:, :, 0]

        title = f"Prediction: {pred_texts[i][::-1]}"
        #print(pred_texts[i]=='')

        ax[i // 16, i % 16].imshow(img, cmap="gray")
        ax[i // 16, i % 16].set_title(title)
        ax[i // 16, i % 16].set_xlabel("label: "+label[::-1], labelpad=1)
        ax[i // 16, i % 16].axis("on")




  plt.show()



In [ ]:
plt.plot(history.history['val_loss'][2:len(history.history['val_loss'])-1])
plt.plot(history.history['loss'][2:len(history.history['loss'])-1]);

In [ ]:
plt.plot(history.history['val_loss'][1:len(history.history['val_loss'])-1])
plt.plot(history.history['loss'][1:len(history.history['loss'])-1])

In [ ]:
plt.plot(history.history['val_loss'])
plt.plot(history.history['loss'])
plt.legend()

##### Iterating through the labels

In [ ]:
i=0
for batch in train_ds:
    batch_label = batch["label"]

    for Label in batch_label:
      indices = tf.gather(Label, tf.where(tf.math.not_equal(Label, padding_token)))
      label = tf.strings.reduce_join(num_to_char(indices))
      label = label.numpy().decode("UTF-8")
      #print(label)
      i+=1
i

In [ ]:
for data in test_ds.take(2):
    images, labels = data["image"], data["label"]

    _, ax = plt.subplots(4, 4, figsize=(15, 8))

    for i in range(16):
        img = images[i]
        img = tf.image.flip_left_right(img)
        img = tf.transpose(img, perm=[1, 0, 2])
        img = (img * 255.0).numpy().clip(0, 255).astype(np.uint8)
        img = img[:, :, 0]

        # Gather indices where label!= padding_token.
        label = labels[i]
        indices = tf.gather(label, tf.where(tf.math.not_equal(label, padding_token)))
        # Convert to string.
        label = tf.strings.reduce_join(num_to_char(indices))
        label = label.numpy().decode("utf-8")

        ax[i // 4, i % 4].imshow(img, cmap="gray")
        ax[i // 4, i % 4].set_title(label[::-1])
        ax[i // 4, i % 4].axis("off")


plt.show()

##### Creating a character accuracy function

Uses the formula:

Character Error Rate (CER) = ( (insertions + subsitutions + deletions) / total character count ) * 100%

In [ ]:
def getCharacterAccuracy(test_ds):
    # for batch in test_ds.take(1):

    totalCharacters = 0
    wrongCharacters = 0
    batchNumber = 0

    for batch in test_ds:
        # batch = tuple(test_ds.take(1))[0]
        batchNumber += 1
        batch_images, batch_labels = batch["image"], batch["label"]
        preds = prediction_model.predict(batch_images,verbose=0)
        pred_texts = decode_batch_predictions(preds)

        input_len = np.ones(preds.shape[0]) * preds.shape[1]
        # Use greedy search. For complex tasks, you can use beam search.
        results = keras.backend.ctc_decode(preds, input_length=input_len, greedy=True)[
            0
        ][0][:, :max_len]

        AllPredictions = []
        # All predicitons
        for res in results:
            res = tf.gather(res, tf.where(tf.math.not_equal(res, -1)))
            res = tf.squeeze(res)
            norRes = tf.Variable(res)

            # print(norRes.shape)
            if len(norRes.shape) == 0:
                norRes = tf.stack((norRes, 99))
                while tf.size(norRes) < 7:
                    norRes = tf.concat((norRes, [99]), axis=-1)
            else:
                while tf.size(norRes) < 7:
                    norRes = tf.concat((norRes, [99]), axis=-1)

            AllPredictions.append(norRes)

        for i in range(len(AllPredictions)):
            #print(batch_labels[i])
            for j in range(7):
                  # print(AllPredictions[i][j])
                  # print(batch_labels[i][j])
                  # print()


                  if AllPredictions[i][j] != batch_labels[i][j]:
                        #print("Entered hell yeah")
                        wrongCharacters +=1
                  totalCharacters+=1
                  #print()
        else:
          continue


    print(wrongCharacters)
    print(totalCharacters)

    errorRate = (wrongCharacters/totalCharacters)
    print("\nError Rate: ",errorRate*100)

    accuracy = 1-errorRate

    print("Accuracy: ",accuracy*100)





# indices = tf.gather(label, tf.where(tf.math.not_equal(label, padding_token)))
# label = tf.strings.reduce_join(num_to_char(indices))
# label = label.numpy().decode("UTF-8")

getCharacterAccuracy(test_ds)


#### Saving current model's weights

In [ ]:
prediction_model.save('currBest2.h5')

# Testing the model after saving

In [ ]:
loaded_model = tf.keras.models.load_model('currBest.h5',compile=False)

In [ ]:
def getCharacterAccuracy2(test_ds):
    # for batch in test_ds.take(1):

    totalCharacters = 0
    wrongCharacters = 0
    batchNumber = 0

    for batch in test_ds:
        # batch = tuple(test_ds.take(1))[0]
        batchNumber += 1
        batch_images, batch_labels = batch["image"], batch["label"]
        preds = loaded_model.predict(batch_images)
        pred_texts = decode_batch_predictions(preds)

        input_len = np.ones(preds.shape[0]) * preds.shape[1]
        # Use greedy search. For complex tasks, you can use beam search.
        results = keras.backend.ctc_decode(preds, input_length=input_len, greedy=False, top_paths=1)[
            0
        ][0][:, :max_len]

        AllPredictions = []
        # All predicitons
        for res in results:
            res = tf.gather(res, tf.where(tf.math.not_equal(res, -1)))
            res = tf.squeeze(res)
            norRes = tf.Variable(res)

            # print(norRes.shape)
            if len(norRes.shape) == 0:
                norRes = tf.stack((norRes, 99))
                while tf.size(norRes) < 7:
                    norRes = tf.concat((norRes, [99]), axis=-1)
            else:
                while tf.size(norRes) < 7:
                    norRes = tf.concat((norRes, [99]), axis=-1)

            AllPredictions.append(norRes)

        for i in range(len(AllPredictions)):
            for j in range(7):
                  #print(AllPredictions[i][j])
                  #print(batch_labels[i][j])

                  if AllPredictions[i][j] != batch_labels[i][j]:
                        #print("Entered hell yeah")
                        wrongCharacters +=1
                  totalCharacters+=1
                  #print()
        else:
          continue


    errorRate = (wrongCharacters/totalCharacters)
    print("Error Rate: ",errorRate*100)

    accuracy = 1-errorRate

    print("Accuracy: ",accuracy*100)





# indices = tf.gather(label, tf.where(tf.math.not_equal(label, padding_token)))
# label = tf.strings.reduce_join(num_to_char(indices))
# label = label.numpy().decode("UTF-8")

getCharacterAccuracy2(test_ds)


In [ ]:
prediction_model = loaded_model

### Miscleanous code

##### Debugging the character counting function

In [ ]:

def getCharacterAccuracy(test_ds):
  #for batch in test_ds.take(1):
  batch = tuple(test_ds.take(1))[0]

  batch_images,batch_labels = batch["image"],batch["label"]
  preds = prediction_model.predict(batch_images)
  pred_texts = decode_batch_predictions(preds)

  input_len = np.ones(preds.shape[0]) * preds.shape[1]
  # Use greedy search. For complex tasks, you can use beam search.
  results = keras.backend.ctc_decode(preds, input_length=input_len, greedy=True)[0][0][
     :, :max_len]

  AllPredictions=[]
  #Get All predicitons for batch
  for res in results:
        res = tf.gather(res, tf.where(tf.math.not_equal(res, -1)))
        res = tf.squeeze(res)
        norRes = tf.Variable(res)

        if len(norRes.shape)==0:
          norRes=tf.stack((norRes,99))
          while tf.size(norRes)<7:
            norRes=tf.concat((norRes,[99]),axis=-1)

        else:
            while tf.size(norRes)<7:
              norRes=tf.concat((norRes,[99]),axis=-1)

        AllPredictions.append(norRes)

  #Plot the batch with the prediction and the label as both arabic and vector notations
  _, ax = plt.subplots(4, 16, figsize=(100, 20))

  for i in range(64):
        img = batch_images[i]
        img = tf.image.flip_left_right(img)
        img = tf.transpose(img, perm=[1, 0, 2])
        img = (img * 255.0).numpy().clip(0, 255).astype(np.uint8)
        img = img[:, :, 0]

        indices = tf.gather(batch_labels[i], tf.where(tf.math.not_equal(batch_labels[i], padding_token)))
        label = tf.strings.reduce_join(num_to_char(indices))
        label = label.numpy().decode("UTF-8")


        ax[i // 16, i % 16].imshow(img, cmap="gray")
        ax[i // 16, i % 16].set_title(str(AllPredictions[i])+"\nPrediction: "+pred_texts[i][::-1])
        ax[i // 16, i % 16].set_xlabel(str(batch_labels[i])+"\nlabel: "+label[::-1], labelpad=1)
        ax[i // 16, i % 16].axis("on")

  plt.show()


getCharacterAccuracy(test_ds)

# Import Libraries

In [ ]:
import cv2 as cv
from google.colab.patches import cv_imshow
import numpy as np

# Open Image

In [ ]:
image = cv.imread('Handwritten arabic text for testing.jpg')

In [ ]:
cv_imshow(image)

# Resize Image

In [ ]:
h, w, c = image.shape #height width what's c? Color channels



new_w = 720
ar = w/h #aspect ratio
new_h = int(new_w/ar)


image = cv.resize(image ,(new_w,new_h), interpolation = cv.INTER_CUBIC)

cv_imshow(image)

# Blur Image

In [ ]:
blurredImage = cv.bilateralFilter(image, 101,30,30)

In [ ]:
cv_imshow(blurredImage)

# Binarization Image

In [ ]:
grayImage = cv.cvtColor(blurredImage, cv.COLOR_BGR2GRAY)

In [ ]:
cv_imshow(grayImage)

In [ ]:
binarizedImage = cv.adaptiveThreshold(grayImage,255,cv.ADAPTIVE_THRESH_MEAN_C,cv.THRESH_BINARY_INV, 17, 20)
cv_imshow(binarizedImage)

# Dilation Segmenting

In [ ]:
kernel = np.ones((20,100), np.uint8) #The number might need to be changed
dialated = cv.dilate(binarizedImage,kernel,iterations= 1)
cv_imshow(dialated)

In [ ]:
(contours,hierarchy) = cv.findContours(dialated.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
#RETR_EXTERNAL means there are no contours within contours
sorted_contours_lines = contours

In [ ]:
lineSegmented = image.copy()

for ctr in sorted_contours_lines:

  if cv.contourArea(ctr) < 1000: #removes noise but can be adjusted
      continue
  x,y,w,h = cv.boundingRect(ctr)
  cv.rectangle(lineSegmented,(x,y),(x+w,y+h),(0,0,250),2)

cv_imshow(lineSegmented)

## Word Segmentation

In [ ]:
kernel = np.ones((25,25), np.uint8) #The number might need to be changed
dialated2 = cv.dilate(binarizedImage,kernel,iterations= 1)
cv_imshow(dialated2)

In [ ]:
img3 = image.copy()
words_list = []

for line in sorted_contours_lines:
  if cv.contourArea(line) < 1000: #removes noise but can be adjusted
      continue
  x,y,w,h = cv.boundingRect(line)
  roi_line = dialated2[y:y+w, x:x+w]
  #cv_imshow(roi_line)

  #Draw contours on each word
  (cnt,hierarchy) = cv.findContours(roi_line.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
  #RETR_EXTERNAL means there are no contours within contours
  sorted_contours_words = sorted(cnt, key = lambda cntr: cv.boundingRect(cntr)[0], reverse=True) #Contains (x,y,w,h) x and y locations and width and height

  for word in sorted_contours_words:

    if cv.contourArea(word) < 600: #removes noise
      continue
    x2,y2,w2,h2 = cv.boundingRect(word)
    words_list.append([x+x2,y+y2,x+x2+w2,y+y2+h2])
    #print([x+x2,y+y2,x+x2+w2,y+y2+h2])
    cv.rectangle(img3,(x+x2,y+y2),(x+x2+w2,y+y2+h2),(255,255,100),2)
    #cv_imshow(img3)


cv_imshow(img3)

### Store each words from top to bottom and right to left.

In [ ]:
words_list = []

for ctr in sorted_contours_lines[::-1]:

  if cv.contourArea(ctr) < 1000: #removes noise but can be adjusted
      continue
  x,y,w,h = cv.boundingRect(ctr)
  line = dialated2 [y:y+h,x:x+w]

  (cnt,hierarchy) = cv.findContours(line.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)

  sorted_contours_words = sorted(cnt, key = lambda cntr: cv.boundingRect(cntr)[0], reverse=True)

  for word in sorted_contours_words:

    if cv.contourArea(word) < 600: #removes noise
      continue
    x2,y2,w2,h2 = cv.boundingRect(word)
    words_list.append([x+x2,y+y2,x+x2+w2,y+y2+h2])




# Store each word separtely

In [ ]:
words=[]

for word in words_list:
  word = image[ word[1]:word[3], word[0]:word[2]]
  words.append(word)
  cv_imshow(word)

# Resize all of the words

In [ ]:
def resizeImage(word):

  word = cv.resize(word,(64,32),interpolation=cv.INTER_CUBIC)

  return word

In [ ]:
resized_words = [ resizeImage(word) for word in words]

In [ ]:
for word in resized_words:
  cv_imshow(word)

# Binarize the Resized words

In [ ]:
def convertToGray(image):

  image = cv.cvtColor(image,cv.COLOR_BGR2GRAY)

  return image

In [ ]:
gray_words = [ convertToGray(word) for word in resized_words]

In [ ]:
for word in gray_words:
  cv_imshow(word)

In [ ]:
def binarizeWords(image):
  image = cv.adaptiveThreshold(image,255,cv.ADAPTIVE_THRESH_MEAN_C,cv.THRESH_BINARY_INV,17, 10)

  return image

In [ ]:
binary_words = [ binarizeWords(word) for word in gray_words]

In [ ]:
for word in binary_words:
  cv_imshow(word)

In [ ]:
# import os

# if not os.path.exists("./temp"):
#   os.mkdir("./temp")

# i=0
# for word in binary_words:
#   cv.imwrite("./temp/"+str(i)+".jpg",word)
#   i+=1

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import StringLookup
from tensorflow import keras

In [ ]:

image = tf.cast(binary_words[2], tf.float32) / 255.0

In [ ]:
image = binary_words[5]

In [ ]:
i=-10

image = tf.cast(binary_words[i], tf.float32) / 255.0
cv_imshow(binary_words[i])

tempImage= tf.transpose(image)
tempImage = np.expand_dims(tempImage,-1)

preds = prediction_model.predict(np.array([tempImage]))
characters = {'ن', 'س', 'ة', 'ك', '.', 'ء', '،', 'ف', 'ه', 'ب', 'خ', 'ؤ', 'غ', 'ز', 'ق', 'ث', 'ح', 'ذ', 'ئ', 'آ', 'إ', 'ض', 'ش', 'م', 'ر', 'ص', 'ا', 'ٍ', ':', 'و', 'أ', 'ظ', 'ل', 'ج', 'د', 'ع', 'ط', 'ت', 'ي'}


char_to_num = StringLookup(vocabulary=list(sorted(characters)), mask_token=None)


num_to_char = StringLookup(
    vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True
)

AUTOTUNE = tf.data.AUTOTUNE


def decode_batch_predictions(pred):
    max_len=7
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    # Use greedy search. For complex tasks, you can use beam search.
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=False)[0][0][
        :, :max_len
    ]
    # Iterate over the results and get back the text.
    output_text = []
    #print(results)
    for res in results:
        res = tf.gather(res, tf.where(tf.math.not_equal(res, -1)))
        res = tf.strings.reduce_join(num_to_char(res)).numpy().decode("UTF-8")
        output_text.append(res)

    return output_text
pred_texts = decode_batch_predictions(preds)
pred_texts